In [1]:
import tensorflow as tf

In [2]:
import os

In [3]:
W = tf.Variable(tf.zeros([4,3]),name="weights")
b = tf.Variable(tf.zeros([3]),name="bias")

In [4]:
def interference(X):
    return tf.nn.softmax(combine_inputs(X))

In [5]:
def combine_inputs(X):
    return tf.matmul(X,W)+b

In [6]:
def loss(X,Y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=combine_inputs(X),logits=Y))

In [7]:
def inputs():
    sepal_length,sepal_width,petal_length,petal_width,label = read_csv(100,"iris.data",[[0.0],[0.0],[0.0],[0.0],[""]])
    
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([tf.equal(label,["Iris-setosa"]),tf.equal(label,["Iris-versicolor"]),tf.equal(label,["Iris-virginica"])])),0))
    
    features = tf.transpose(tf.stack([sepal_length,sepal_width,petal_length,petal_width]))
    
    return features,label_number

In [8]:
def evaluate(sess,X,Y):
    predicted = tf.cast(tf.arg_max(interference(X),1),tf.int32)
    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted,Y),tf.float32))))

In [9]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [10]:
def read_csv(batch_size,file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.dirname("__file__") + "/Users/suhyeongcho/Desktop/Github/tf-notebooks/" + file_name])
    reader = tf.TextLineReader(skip_header_lines = 1)
    key, value = reader.read(filename_queue)
    
    decoded = tf.decode_csv(value,record_defaults=record_defaults)
    
    return tf.train.shuffle_batch(decoded,batch_size=batch_size,capacity=batch_size*50,min_after_dequeue=batch_size)

In [11]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    X,Y=inputs()
    
    total_loss = loss(X,Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    
    training_steps = 1000
    
    for step in range(training_steps):
        sess.run([train_op])
        
        if step % 10 == 0:
            print("loss : ",sess.run([total_loss]))
            
    evaluate(sess,X,Y)
    
    coord.request_stop()
    coord.join(threads)
    
    sess.close()

TypeError: Value passed to parameter 'features' has DataType int32 not in list of allowed values: float16, float32, float64